In [16]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets
from sklearn.preprocessing import scale
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
import pandas as pd

In [2]:
train_data = datasets.load_boston()

In [3]:
type(train_data)

sklearn.datasets.base.Bunch

In [4]:
features = train_data.feature_names
print features

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [5]:
# obtain features and target
features = train_data.data
target = train_data.target


In [6]:
# scaling features
features = scale(features)


In [7]:

param_set = np.linspace(1,10,num=200)
print param_set[:10]

[ 1.          1.04522613  1.09045226  1.13567839  1.18090452  1.22613065
  1.27135678  1.31658291  1.36180905  1.40703518]


In [8]:
def param_iterator(param_list, model=KNeighborsRegressor):
    '''takes the list of parameters to iterate over and create regressors w/ those parameters'''
    
    model_list = {}
    for elem in param_list:
        regressor = model(n_neighbors=5, weights='distance', p=elem)
        model_list.update({elem: regressor})
    
    return model_list



In [9]:
# build the list of KNR with different p's
knr_list = param_iterator(param_set)


In [13]:
def get_crossValScore(model_list):
    '''takes the model w/ different parameters and evaluates the cross validation score'''
    model_list_keys = model_list.keys()
    model_list_values = model_list.values()
    
    score_list = []
    
    for model in model_list_values:
        score = cross_val_score(model, features, y=target, scoring='mean_squared_error')
        score_list.append(score)
        
    #score_dict = {key:value for key,value in model_list.iteritems()}
    #model_list_items = model_list.viewitems()
    
    score_dict = dict(zip(model_list_keys, score_list))      
      
    return score_dict
        
#validator = cross_val_score(kn_regressor, features, scoring='mean_squared_error')
#model_accuracy = KFold(n_folds=5, random_state=42, shuffle=True)

In [41]:
scores_params = get_crossValScore(knr_list)
#print score_list


In [42]:
# create new dataframe with parameters-score
param_score_data = pd.DataFrame.from_dict(scores_params, orient='index')
param_score_data.head()

,0,1,2
1.000000,-28.249489,-38.396370,-40.694440
3.577889,-22.319982,-56.298715,-43.269372
7.467337,-25.521623,-65.727381,-48.131459
4.437186,-23.121002,-62.689367,-47.601396
2.311558,-22.851286,-42.944868,-40.857731


In [54]:
score_mean = param_score_data.mean(axis=1)
score_mean_max = score_mean.max()

# get the id of the score with maximal mean in the array
score_mean_max_id = score_mean.idxmax()

print 'Maximal mean score is %(score)f with the score of %(param)f' % {'score':score_mean_max, 'param':score_mean_max_id}

Maximal mean score is -33.411038 with the score of 1.723618
